In [ ]:
!pwd
![ -d chroma-instance ] || git clone https://github.com/owen8877/chroma-instance/
!cd chroma-instance/chroma_instance/source && ./download_model.sh

/content


In [ ]:
# Copy the backed up weights into working directory
from google.colab import drive
drive.mount('/content/drive')

![ -d '/content/chroma-instance/chroma_instance/weights' ] || mkdir '/content/chroma-instance/chroma_instance/weights'
# !cp '/content/drive/MyDrive/colab_tmp/weights' '/content/chroma-instance/chroma_instance/' -r
# !cp '/content/drive/MyDrive/colab_tmp/weights/instcolorization' '/content/chroma-instance/chroma_instance/weights/' -r
# !cp '/content/drive/MyDrive/colab_tmp/weights/chroma_gan' '/content/chroma-instance/chroma_instance/weights/' -r
# !cp '/content/drive/MyDrive/colab_tmp/weights/mask_rcnn' '/content/chroma-instance/chroma_instance/weights/' -r
![ -f '/content/chroma-instance/chroma_instance/dataset/train2017.zip' ] || cp '/content/drive/MyDrive/colab_tmp/dataset/train2017.zip' '/content/chroma-instance/chroma_instance/dataset/' -r
!if [ ! -d '/content/chroma-instance/chroma_instance/dataset/train' ]; then \
    pushd '/content/drive/MyDrive/colab_tmp/dataset/'; \
    unzip train2017.zip; \
    mv train2017 train; \
    popd; \
fi

In [ ]:
import sys
sys.path.append('/content/chroma-instance/chroma_instance/source')

%tensorflow_version 1.x
import tensorflow as tf

TensorFlow 1.x selected.


## Chroma Instance

In [ ]:
cd /content/chroma-instance/chroma_instance/source/chroma_instance

In [ ]:
from chroma_instance import model

model.train()